# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 
import re

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt


def clean_non_western_encoded_characters_commands(text: str) -> str:
    """ Remove non-western encoded characters from a string
    List may need to grow.
    
    :param text: the text to clean
    :return: the cleaned text
    """
    text = re.sub(r"(\\begin{CJK}{UTF8}{gbsn})(.*?)(\\end{CJK})", r"\2", text)
    return text


def get_initials(name: str) -> str:
    """ Get the short name, e.g., A.-B. FamName
    :param name: full name
    :returns: initials
    """
    initials = []
    # account for non western names often in ()
    if '(' in name:
        name = clean_non_western_encoded_characters_commands(name)
        suffix = re.findall(r"\((.*?)\)", name)[0]
        name = name.replace(f"({suffix})", '')
    else:
        suffix = ''
    split = name.split()
    for token in split[:-1]:
        if '-' in token:
            current = '-'.join([k[0] + '.' for k in token.split('-')])
        else:
            current = token[0] + '.'
        initials.append(current)
    initials.append(split[-1].strip())
    if suffix:
        initials.append(f"({suffix})")
    return ' '.join(initials)

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

def robust_call(fn, value, *args, **kwargs):
    try:
        return fn(value, *args, **kwargs)
    except Exception:
        return value

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [robust_call(mpia.get_initials, k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

P. Mollière  ->  P. Mollière  |  ['P. Mollière']
E. Matthews  ->  E. Matthews  |  ['E. Matthews']
T. Henning  ->  T. Henning  |  ['T. Henning']
T. Trifonov  ->  T. Trifonov  |  ['T. Trifonov']
T. Henning  ->  T. Henning  |  ['T. Henning']
T. Trifonov  ->  T. Trifonov  |  ['T. Trifonov']
J. Li  ->  J. Li  |  ['J. Li']


J. Liu  ->  J. Liu  |  ['J. Liu']
M. Clausen  ->  M. Clausen  |  ['M. Clausen']
Arxiv has 76 new papers today
          6 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print("Issues with the citations")
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/6 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2410.10933


extracting tarball to tmp_2410.10933...

 done.


P. Mollière  ->  P. Mollière  |  ['P. Mollière']
E. Matthews  ->  E. Matthews  |  ['E. Matthews']
T. Henning  ->  T. Henning  |  ['T. Henning']


...

 done.
Retrieving document from  https://arxiv.org/e-print/2410.11611


extracting tarball to tmp_2410.11611...

 done.
Retrieving document from  https://arxiv.org/e-print/2410.11808


extracting tarball to tmp_2410.11808...

 done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2410.10933-b31b1b.svg)](https://arxiv.org/abs/2410.10933) | **Water depletion and 15NH3 in the atmosphere of the coldest brown dwarf observed with JWST/MIRI**  |
|| H. Kühnle, et al. -- incl., <mark>P. Mollière</mark>, <mark>E. Matthews</mark>, <mark>T. Henning</mark> |
|*Appeared on*| *2024-10-16*|
|*Comments*| *Submitted to A&A, 29 pages, 21 figures*|
|**Abstract**|            With a temperature of $\sim 285$ K WISE0855 is the coldest brown dwarf observed so far. Using the James Webb Space Telescope (JWST) we obtained observations that allow us to characterize WISE0855s atmosphere focusing on vertical variation in the water steam abundance, measuring trace gas abundances and receiving bulk parameters for this cold object. We observed the ultra cool dwarf WISE0855 using the Mid-Infrared Instrument Medium Resolution Spectrometer (MIRI/MRS) onboard JWST at a spectral resolution of up to 3750. We combined the observation with published data from the Near Infrared Spectrograph (NIRSpec) G395M and PRISM modes yielding a spectrum ranging from 0.8 to 22 um. We apply atmospheric retrievals using petitRADTRANS to measure atmospheric abundances, the pressure-temperature structure, radius and gravity of the brown dwarf. We also employ publicly available clear and cloudy self-consistent grid models to estimate bulk properties of the atmosphere such as the effective temperature, radius, gravity and metallicity. Atmospheric retrievals constrain a variable water abundance profile in the atmosphere, as predicted by equilibrium chemistry. We detect the 15NH3 isotopologue and infer a ratio of mass fraction of 14NH3/15NH3 = 332+63-43 for the clear retrieval. We measure the bolometric luminosity by integrating the presented spectrum and obtain a value of log(L/L$_{\odot}$) = -7.291+/-0.008. The detected water depletion indicates that water condenses out in the upper atmosphere due to the very low effective temperature of WISE0855. The height in the atmosphere where this occurs is covered by the MIRI/MRS data, and thus demonstrates the potential of MIRI to characterize cold gas giants atmospheres. Comparing the data to retrievals and self-consistent grid models, we do not detect signs for water ice clouds, although their spectral features have been predicted in previous studies.         |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2410.11436-b31b1b.svg)](https://arxiv.org/abs/2410.11436) | **Search for young stellar objects within 4XMM-DR13 using CatBoost and SPE**  |
|| X. Ma, et al. -- incl., <mark>J. Li</mark> |
|*Appeared on*| *2024-10-16*|
|*Comments*| *14 pages, 22 figures, 3 tables, accepted by the Astronomical Journal*|
|**Abstract**|            Classifying and summarizing large data sets from different sky survey projects is essential for various subsequent scientific research. By combining data from 4XMM-DR13, SDSS DR18, and CatWISE, we formed an XMM-WISE-SDSS sample that included information in the X-ray, optical, and infrared bands. By cross-matching this sample with datasets from known spectral classifications from SDSS and LAMOST, we obtained a training dataset containing stars, galaxies, quasars, and Young Stellar Objects (YSOs). Two machine learning methods, CatBoost and Self-Paced Ensemble (SPE), were used to train and construct machine learning models through training sets to classify the XMM-WISE-SDSS sample. Notably, the SPE classifier showed excellent performance in YSO classification, identifying 1102 YSO candidates from 160,545 sources, including 258 known YSOs. Then we further verify whether these candidates are YSOs by the spectra in LAMOST and the identification in the SIMBAD and VizieR dabtabases. Finally there are 412 unidentified YSO candidates. The discovery of these new YSOs is an important addition to existing YSO samples and will deepen our understanding of star formation and evolution. Moreover we provided a classification catalog for the whole XMM-WISE-SDSS sample.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: '69117' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2410.11611-b31b1b.svg)](https://arxiv.org/abs/2410.11611) | **Predicting photospheric UV emission from stellar evolutionary models**  |
|| S. Wang, X. Li, H. Han, <mark>J. Liu</mark> |
|*Appeared on*| *2024-10-16*|
|*Comments*| *10 pages, 6 figures, 3 tables. Accepted for publication in ApJ*|
|**Abstract**|            Stellar ultraviolet (UV) emission serves as a crucial indicator for estimating magnetic activity and evaluating the habitability of exoplanets orbiting stars. In this paper, we present a straightforward method to derive stellar photospheric UV emission for F to M main-sequence stars. By using PARSEC models, we establish relations between near-UV (NUV) and far-UV (FUV) magnitudes from the Galaxy Evolution Explorer (GALEX), NUV magnitudes from the China Space Station Telescope, and stellar effective temperatures and Gaia BP$-$RP color for different metallicities. Together with the observed sample, we find that for NUV emission, the photospheric contribution to the observed flux is less than 20% for M stars, around 10% to 70% for G stars, and ranges from 30% to 85% for G and F stars. For FUV emission, the photospheric contribution is less than $10^{-6}$ for M stars, below $10^{-4}$ for K stars, around $10^{-4}$ to 10% for G stars, and between 6% and 50% for F stars. Our work enables the simple and effective determination of stellar excess UV emission and the exploration of magnetic activity.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2410.11808-b31b1b.svg)](https://arxiv.org/abs/2410.11808) | **The JWST Emission Line Survey (JELS): An untargeted search for H$\alpha$ emission line galaxies at $z > 6$ and their physical properties**  |
|| C. A. Pirie, et al. -- incl., <mark>M. Clausen</mark> |
|*Appeared on*| *2024-10-16*|
|*Comments*| *Submitted to MRNAS; 27 pages, 19 figures. Accompanying data will be publicly released upon acceptance of this manuscript*|
|**Abstract**|            We present the first results of the JWST Emission Line Survey (JELS). Utilising the first NIRCam narrow-band imaging at 4.7$\mu$m, over 63 arcmin$^{2}$ in the PRIMER/COSMOS field, we have identified 609 emission line galaxy candidates. From these, we robustly selected 35 H$\alpha$ star-forming galaxies at $z \sim 6.1$, with H$\alpha$ star-formation rates ($\rm{SFR_{H\alpha}}$) of $\sim0.9-15\ \rm{M_{\odot} yr^{-1}}$. Combining our unique H$\alpha$ sample with the exquisite panchromatic data in the field, we explore their physical properties and star-formation histories, and compare these to a broad-band selected sample at $z\sim 6$ to offer vital new insight into the nature of high-redshift galaxies. UV-continuum slopes ($\beta$) are considerably redder for our H$\alpha$ sample ($\langle\beta\rangle\sim-1.92$) compared to the broad-band sample ($\langle\beta\rangle\sim-2.35$). This is not due to dust attenuation as our H$\alpha$ sample is relatively dust-poor (median $A_V=0.23$); instead, we argue that the reddened slopes could be due to nebular continuum. We compared $\rm{SFR_{H\alpha}}$ and the UV-continuum-derived $\rm{SFR_{UV}}$ to SED-fitted measurements averaged over canonical timescales of 10 and 100 Myr ($\rm{SFR_{10}}$ and $\rm{SFR_{100}}$). We find an increase in recent SFR for our sample of H$\alpha$ emitters, particularly at lower stellar masses ($<10^9 \rm{M_{\odot}}$). We also find that $\rm{SFR_{H\alpha}}$ strongly traces SFR averaged over 10 Myr timescales, whereas the UV-continuum over-predicts SFR on 100 Myr timescales at low stellar masses. These results point to our H$\alpha$ sample undergoing `bursty' star formation. Our F356W $z \sim 6$ sample show a larger scatter in $\rm{SFR_{10}/SFR_{100}}$ across all stellar masses, highlighting how narrow-band photometric selections of H$\alpha$ emitters are key to quantifying the burstiness of star-formation activity.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2410.11424-b31b1b.svg)](https://arxiv.org/abs/2410.11424) | **Radial velocity technique**  |
|| <mark>T. Trifonov</mark> |
|*Appeared on*| *2024-10-16*|
|*Comments*| *Preprint of a chapter for the 'Encyclopedia of Astrophysics' (Editor-in-Chief Ilya Mandel, Section Editor Dimitri Veras) to be published by Elsevier as a Reference Module. 23 pages, 14 Figures, 2 Tables*|
|**Abstract**|            The precise Doppler method for measuring stellar radial velocities (RV) is a fundamental technique in modern astronomy. This method records a star's spectrum and detects periodic Doppler shifts in its spectral features, which indicate the gravitational influences induced by orbiting companions. The Doppler technique has yielded remarkable successes in exoplanet detection, uncovering a diverse array of planetary systems ranging from hot Jupiters to Neptune-mass planets and super-Earths. Having led to the discovery of over 1100 exoplanets, the RV method is the most effective approach for measuring orbital geometries and minimum masses, which are of fundamental importance when accessing planet formation and evolution scenarios. However, exoplanet detection via precise RV measurements poses significant challenges, including managing various sources of interference, such as instrumental errors, and mitigating spurious Doppler shifts induced by phenomena like stellar activity. Key to this technique's advancement is instrumental calibration methods, notably precise calibration methods and ultra-stable spectrographs. This technique holds promise in systematically exploring the domain of Jovian analogs, rocky and icy planets within the habitable zones of their parent stars, and providing crucial follow-up observations for transiting candidates detected by space missions. The synergy between transit and Doppler measurements of exoplanets, when feasible, has provided a comprehensive set of orbital and physical parameters for exoplanets, such as the dynamical mass and mean planet density, which is enhancing our understanding of their internal composition. Additionally, ongoing efforts aim to improve the RV technique further by developing more stable calibration techniques aimed at detecting Earth-like analogs around Solar-type stars that require cm/s RV precision.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Planck' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2410.11037-b31b1b.svg)](https://arxiv.org/abs/2410.11037) | **TESS Giants Transiting Giants. VII. A Hot Saturn Orbiting an Oscillating Red Giant Star**  |
|| N. Saunders, et al. -- incl., <mark>T. Trifonov</mark>, <mark>T. Henning</mark> |
|*Appeared on*| *2024-10-16*|
|*Comments*| *16 pages, 9 figures, 3 tables*|
|**Abstract**|            We present the discovery of TOI-7041 b (TIC 201175570 b), a hot Saturn transiting a red giant star with measurable stellar oscillations. We observe solar-like oscillations in TOI-7041 with a frequency of maximum power of $\nu_{\rm max} = 218.50\pm2.23$ $\mu$Hz and a large frequency separation of $\Delta\nu = 16.5282\pm0.0186$ $\mu$Hz. Our asteroseismic analysis indicates that TOI-7041 has a radius of $4.10 \pm 0.06$(stat) $\pm$ 0.05(sys) $R_\odot$, making it one of the largest stars around which a transiting planet has been discovered with the Transiting Exoplanet Survey Satellite (TESS), and the mission's first oscillating red giant with a transiting planet. TOI-7041 b has an orbital period of $9.691 \pm 0.006$ days and a low eccentricity of $e = 0.04 \pm 0.04$. We measure a planet radius of $1.02 \pm 0.03$ $R_J$ with photometry from TESS, and a planet mass of $0.36 \pm 0.16$ $M_J$ ($114 \pm 51$ $M_\oplus$) with ground-based radial velocity measurements. TOI-7041 b appears less inflated than similar systems receiving equivalent incident flux, and its circular orbit indicates that it is not undergoing tidal heating due to circularization. The asteroseismic analysis of the host star provides some of the tightest constraints on stellar properties for a TESS planet host and enables precise characterization of the hot Saturn. This system joins a small number of TESS-discovered exoplanets orbiting stars that exhibit clear stellar oscillations and indicates that extended TESS observations of evolved stars will similarly provide a path to improved exoplanet characterization.         |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error list index out of range</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2410.10933/./DetEmpError.png', 'tmp_2410.10933/./CompCloudy.png', 'tmp_2410.10933/./Isotopologue.png']
copying  tmp_2410.10933/./DetEmpError.png to _build/html/
copying  tmp_2410.10933/./CompCloudy.png to _build/html/
copying  tmp_2410.10933/./Isotopologue.png to _build/html/
exported in  _build/html/2410.10933.md
    + _build/html/tmp_2410.10933/./DetEmpError.png
    + _build/html/tmp_2410.10933/./CompCloudy.png
    + _build/html/tmp_2410.10933/./Isotopologue.png


## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\todo}[1]{\textbf{\color{orange}#1}}$
$\newcommand{\RJ}{R_\mathrm{J}}$
$\newcommand{\MJ}{M_\mathrm{J}}$</div>



<div id="title">

# Water depletion and $\mathrm{^{15}NH_3}$ in the atmosphere of the coldest brown dwarf observed with JWST/MIRI

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2410.10933-b31b1b.svg)](https://arxiv.org/abs/2410.10933)<mark>Appeared on: 2024-10-16</mark> -  _Submitted to A&A, 29 pages, 21 figures_

</div>
<div id="authors">

H. Kühnle, et al. -- incl., <mark>P. Mollière</mark>, <mark>E. Matthews</mark>, <mark>T. Henning</mark>

</div>
<div id="abstract">

**Abstract:** With a temperature of $\sim$ 285 K WISE 0855 is the coldest brown dwarf observed so far. Such cold gas giants enable probing atmospheric physics and chemistry of evolved objects similar to the Solar System gas giants. Using the James Webb Space Telescope (JWST) we obtained observations that allow us to characterize WISE 0855's atmosphere focusing on vertical variation in the water steam abundance, measuring trace gas abundances and receiving bulk parameters for this cold object. We observed the ultra cool dwarf WISE 0855 using the Mid-Infrared Instrument Medium Resolution Spectrometer (MIRI/MRS) onboard JWST at a spectral resolution of up to 3'750. We combined the observation with the published data from the Near Infrared Spectrograph (NIRSpec) G395M and PRISM modes yielding a spectrum ranging from 0.8 to 22 µm. We applied atmospheric retrievals using \texttt{petitRADTRANS} to measure atmospheric abundances, the pressure-temperature structure, radius and gravity of the brown dwarf. We also employed publicly available clear and cloudy self-consistent grid models to estimate bulk properties of the atmosphere such as the effective temperature, radius, gravity and metallicity. Atmospheric retrievals constrain a variable water abundance profile in the atmosphere, as predicted by equilibrium chemistry. We detect the $\mathrm{^{15}NH_3}$ isotopologue and infer a ratio of mass fraction of $\mathrm{^{14}NH_3}$ / $\mathrm{^{15}NH_3}$ = 332 $^{+63}_{-43}$ for the clear retrieval. We measure the bolometric luminosity by integrating the presented spectrum and obtain a value of $\mathrm{log(L/\textup{L}_\odot)} = -7.291 \pm 0.008$ . The detected water depletion indicates that water condenses out in the upper atmosphere due to the very low effective temperature of WISE 0855. The height in the atmosphere where this occurs is covered by the MIRI/MRS data, and thus demonstrates the potential of MIRI to characterize cold gas giant's atmospheres. Comparing the data to retrievals and self-consistent grid models, we do not detect signs for water ice clouds, although their spectral features have been predicted in previous studies.

</div>

<div id="div_fig1">

<img src="tmp_2410.10933/./DetEmpError.png" alt="Fig5" width="100%"/>

**Figure 5. -** Resulting cube images of channels 1 to 4 with sub channels A to C after dither subtraction. The wavelengths are chosen such that they refer to the detector image with the highest flux at the source in each band. The red circle shows the aperture for the flux extraction of the spectrum at one FWHM of the PSF of the source. The blue circle refers to three times the FWHM on which nine black circles of one FWHM are placed. We extract the flux from each circle and measure the variability to obtain an error including the noise in the background, $\sigma_{emp}$.  (*fig:errorestimation*)

</div>
<div id="div_fig2">

<img src="tmp_2410.10933/./CompCloudy.png" alt="Fig9" width="100%"/>

**Figure 9. -** Best fit grid model spectra for the clear ATMO++ in red, the thick cloudy non-equilibrium model by Lacy2023 in blue and and the clear non-equilibrium in orange. Subplot a) shows an inset of the PRISM data, which has a significant lower amount of flux. Subfigure b) gives the NIRSpec/G395M and MIRI/MRS data set. Subplot c) and d) show insets in panel b) on the NIRspec/G395M region and the prominent $NH_3$ absorption feature. The corresponding $\chi^2$ values are 2.57, 1.97 and 5.66 for ATMO++, the Lacy cloudy (thick) and clear non-equilibrium model fits. As for the retrievals and for better visibility we re-bined the models and data to a resolution R = 500 for the NIRSpec/G395M and MIRI/MRS and kept NIRSpec/PRISM at R = 100. (*fig:cloud_self*)

</div>
<div id="div_fig3">

<img src="tmp_2410.10933/./Isotopologue.png" alt="Fig11" width="100%"/>

**Figure 11. -** We show the best fit spectrum with $^{15}$$NH_3$ included in blue, the best fit removing the $^{15}$$NH_3$ opacity in red and in black the data with error bars. The top right panel b) shows the $^{15}$$NH_3$ feature observed by the MIRI/MRS spectrum of WISE 0855 with an inset shown in panel a). The middle plot d) shows the residuals of the fit with and without $^{15}$$NH_3$ in blue and red respectively. The lowest plot f) shows the $^{15}$$NH_3$ opacity used for the retrieval at 277 K and for the pressures of the clear retrieval with varying $H_2$O profile. Insets of the middle and lower plots are shown in panels c) and e) (*fig:isotop*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2410.10933"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

220  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

12  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

4  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
